In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from features.settings import Baseline_Settings, Pair_Hinge_Settings, Pair_Log_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings
from features.models import ConvModelSmall, ConvModelMedium, ConvModelLarge
from features.models import GCNNModelSmall, GCNNModelMedium
from classes.models import LinearClassifier
from datasets import load_cifar, load_cifar10, load_cifar100
from sklearn.svm import LinearSVC
import time

/home/dqi/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/dqi/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data, samplers, pair_samplers, triplet_samplers = load_cifar10(augment=True)

CIFAR-10 loaded in 115 seconds


In [3]:
def test_model(model, settings, data, feature_samplers, classification_samplers, num_features, num_steps, lr, keep_prob, reg):
    tf.reset_default_graph()
    f_train, f_test = feature_samplers
    c_train, c_test = classification_samplers
    (x_train, y_train), (x_test, y_test) = data
    _, H, W, C = x_train.shape
    
    x = tf.placeholder(tf.float32, shape=[None, H, W, C])
    y = tf.placeholder(tf.int64, shape=[None])
    dropout = tf.placeholder(tf.float32)
    training = tf.placeholder(tf.bool)
    m = model(x=x, y=y, settings=settings, num_chan=C, num_features=num_features, 
              lr=lr, reg=reg, dropout=dropout, training=training)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        print("begin training features // num_features: %g, lr: %g, reg: %g"%(num_features, lr, reg))
        train_time = time.time()
        for step in range(num_steps):        
            x_, y_ = f_train.sample(200)
            sess.run(m.optimize, feed_dict={x:x_, y:y_, dropout:keep_prob, training:True})     
            if step % 4000 == 0:
                train_loss, train_acc = sess.run([m.loss, m.acc], feed_dict={x:x_, y:y_, dropout:1.0, training:False})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training features // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        x_, y_ = f_test.sample(5000)
        test_error = 1 - sess.run(m.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        svc = LinearSVC(random_state=0)
        x_, y_ = c_train.sample(20000)
        features_train = sess.run(m.features, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        svc.fit(features_train, y_)
        features_test = sess.run(m.features, feed_dict={x:x_test, y:y_test, dropout:1.0, training:False})
        print("classification accuracy: {:.4f}\n".format(svc.score(features_test, y_test)))

# CIFAR 10

In [5]:
test_model(model=GCNNModelSmall, settings=Baseline_Settings, 
           data=data['cifar10'], feature_samplers=samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=40000, lr=0.07, keep_prob=0.5, reg=0)

begin training features // num_features: 10, lr: 0.07, reg: 0
	step 0: train loss 2.27743, train error 0.915
	step 4000: train loss 0.995881, train error 0.365
	step 8000: train loss 0.956203, train error 0.345
	step 12000: train loss 0.93045, train error 0.35
	step 16000: train loss 0.787815, train error 0.285
	step 20000: train loss 0.527499, train error 0.185
	step 24000: train loss 0.576886, train error 0.205
	step 28000: train loss 0.556494, train error 0.2
	step 32000: train loss 0.487627, train error 0.145
	step 36000: train loss 0.476234, train error 0.175
end training features // time elapsed: 741.2900 s
test set error: 0.1952 // time elapsed: 0.2801 s
classification accuracy: 0.8026



In [4]:
test_model(model=GCNNModelSmall, settings=Baseline_Settings, 
           data=data['cifar10'], feature_samplers=samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=40000, lr=0.07, keep_prob=0.7, reg=0)

begin training features // num_features: 10, lr: 0.07, reg: 0
	step 0: train loss 2.40137, train error 0.885
	step 4000: train loss 0.809548, train error 0.325
	step 8000: train loss 0.435607, train error 0.135
	step 12000: train loss 0.305689, train error 0.1
	step 16000: train loss 0.208463, train error 0.06
	step 20000: train loss 0.104655, train error 0.015
	step 24000: train loss 0.068234, train error 0.00999999
	step 28000: train loss 0.0471136, train error 0.005
	step 32000: train loss 0.0297929, train error 0
	step 36000: train loss 0.0248514, train error 0.005
end training features // time elapsed: 785.9561 s
test set error: 0.1888 // time elapsed: 0.3274 s
classification accuracy: 0.8188



In [5]:
test_model(model=GCNNModelSmall, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=50000, lr=0.05, keep_prob=0.7, reg=0)

begin training features // num_features: 10, lr: 0.05, reg: 0
	step 0: train loss 6.03827, train error 0.58
	step 4000: train loss 0.441758, train error 0.225
	step 8000: train loss 0.366362, train error 0.17
	step 12000: train loss 0.220132, train error 0.09
	step 16000: train loss 0.363013, train error 0.13
	step 20000: train loss 0.193634, train error 0.06
	step 24000: train loss 0.19811, train error 0.075
	step 28000: train loss 0.110381, train error 0.045
	step 32000: train loss 0.099821, train error 0.045
	step 36000: train loss 0.121917, train error 0.06
	step 40000: train loss 0.135041, train error 0.05
	step 44000: train loss 0.130919, train error 0.055
	step 48000: train loss 0.160531, train error 0.075
end training features // time elapsed: 2334.3376 s
test set error: 0.0678 // time elapsed: 1.1123 s
classification accuracy: 0.7859



In [6]:
test_model(model=GCNNModelSmall, settings=Pair_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=pair_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=20000, lr=0.002, keep_prob=0.8, reg=0)

begin training features // num_features: 10, lr: 0.002, reg: 0
	step 0: train loss 2.43421, train error 0.89
	step 4000: train loss 1.00583, train error 0.51
	step 8000: train loss 0.972453, train error 0.45
	step 12000: train loss 0.914387, train error 0.44
	step 16000: train loss 0.961901, train error 0.45
end training features // time elapsed: 656.4830 s
test set error: 0.4436 // time elapsed: 0.4432 s
classification accuracy: 0.2872



In [ ]:
test_model(model=GCNNModelSmall, settings=Pair_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=pair_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=20000, lr=0.001, keep_prob=0.8, reg=0)

In [8]:
test_model(model=ConvModelSmall, settings=Baseline_Settings, 
           data=data['cifar10'], feature_samplers=samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=40000, lr=0.035, keep_prob=0.6, reg=0)

begin training features // num_features: 10, lr: 0.035, reg: 0
	step 0: train loss 2.34671, train error 0.91
	step 4000: train loss 1.53022, train error 0.55
	step 8000: train loss 1.33492, train error 0.465
	step 12000: train loss 1.27821, train error 0.485
	step 16000: train loss 1.31745, train error 0.47
	step 20000: train loss 1.29156, train error 0.5
	step 24000: train loss 1.24193, train error 0.48
	step 28000: train loss 1.20333, train error 0.49
	step 32000: train loss 1.23702, train error 0.44
	step 36000: train loss 1.06758, train error 0.395
end training features // time elapsed: 331.7826 s
test set error: 0.3920 // time elapsed: 0.1231 s
classification accuracy: 0.6131



In [9]:
test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=40000, lr=0.035, keep_prob=0.8, reg=0)

begin training features // num_features: 10, lr: 0.035, reg: 0
	step 0: train loss 0.994673, train error 0.455
	step 4000: train loss 0.642121, train error 0.265
	step 8000: train loss 0.514736, train error 0.215
	step 12000: train loss 0.552468, train error 0.23
	step 16000: train loss 0.442118, train error 0.16
	step 20000: train loss 0.37763, train error 0.165
	step 24000: train loss 0.449363, train error 0.2
	step 28000: train loss 0.393079, train error 0.16
	step 32000: train loss 0.407762, train error 0.185
	step 36000: train loss 0.292668, train error 0.135
end training features // time elapsed: 683.7065 s
test set error: 0.1628 // time elapsed: 0.6199 s
classification accuracy: 0.5940



In [ ]:
test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=50000, lr=0.035, keep_prob=0.8, reg=0)

In [7]:
test_model(model=ConvModelMedium, settings=Baseline_Settings, 
           data=data['cifar10'], feature_samplers=samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=40000, lr=0.05, keep_prob=0.7, reg=0)

begin training features // num_features: 10, lr: 0.05, reg: 0
	step 0: train loss 2.33664, train error 0.925
	step 4000: train loss 0.914061, train error 0.33
	step 8000: train loss 0.762235, train error 0.245
	step 12000: train loss 0.695487, train error 0.24
	step 16000: train loss 0.5237, train error 0.185
	step 20000: train loss 0.38993, train error 0.16
	step 24000: train loss 0.345014, train error 0.125
	step 28000: train loss 0.350183, train error 0.115
	step 32000: train loss 0.266992, train error 0.11
	step 36000: train loss 0.324013, train error 0.125
end training features // time elapsed: 607.5023 s
test set error: 0.1974 // time elapsed: 0.3194 s
classification accuracy: 0.8053



In [ ]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=50000, lr=0.055, keep_prob=0.7, reg=0)

begin training features // num_features: 10, lr: 0.055, reg: 0
	step 0: train loss 0.961764, train error 0.435
	step 4000: train loss 0.470443, train error 0.21
	step 8000: train loss 0.440299, train error 0.19
	step 12000: train loss 0.343401, train error 0.15
	step 16000: train loss 0.379978, train error 0.145
	step 20000: train loss 0.276982, train error 0.11
	step 24000: train loss 0.234762, train error 0.085
	step 28000: train loss 0.257317, train error 0.1
	step 32000: train loss 0.233478, train error 0.105
	step 36000: train loss 0.260139, train error 0.11
	step 40000: train loss 0.211513, train error 0.08


In [4]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=50000, lr=0.05, keep_prob=0.7, reg=0)

begin training features // num_features: 10, lr: 0.06, reg: 0
	step 0: train loss 1.74911, train error 0.485
	step 4000: train loss 0.543454, train error 0.235
	step 8000: train loss 0.431922, train error 0.185
	step 12000: train loss 0.48555, train error 0.19
	step 16000: train loss 0.296074, train error 0.105
	step 20000: train loss 0.25537, train error 0.09
	step 24000: train loss 0.258988, train error 0.115
	step 28000: train loss 0.243715, train error 0.11
	step 32000: train loss 0.229151, train error 0.095
	step 36000: train loss 0.206, train error 0.07
	step 40000: train loss 0.222944, train error 0.09
	step 44000: train loss 0.147414, train error 0.06
	step 48000: train loss 0.2713, train error 0.125
end training features // time elapsed: 1905.8077 s
test set error: 0.1064 // time elapsed: 0.9640 s
classification accuracy: 0.7307



In [ ]:
test_model(model=ConvModelMedium, settings=Baseline_Settings, 
           data=data['cifar10'], feature_samplers=samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=50000, lr=0.045, keep_prob=0.7, reg=0)

# CIFAR-100

In [ ]:
test_model(model=ConvModelLarge, settings=Triplet_Hinge_Settings, 
           data=data['cifar100'], feature_samplers=triplet_samplers['cifar100'],
           classification_samplers=samplers['cifar100'],
           num_features=10, num_steps=15000, lr=0.01, keep_prob=0.8, reg=0)

# Past experiment results

In [10]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=20000, lr=0.07, keep_prob=0.8, reg=0)

begin training features // num_features: 10, lr: 0.07, reg: 0
	step 0: train loss 0.987092, train error 0.465
	step 1000: train loss 0.514236, train error 0.205
	step 2000: train loss 0.539344, train error 0.23
	step 3000: train loss 0.525026, train error 0.22
	step 4000: train loss 0.414221, train error 0.18
	step 5000: train loss 0.490698, train error 0.21
	step 6000: train loss 0.380224, train error 0.17
	step 7000: train loss 0.337057, train error 0.125
	step 8000: train loss 0.303571, train error 0.135
	step 9000: train loss 0.45559, train error 0.175
	step 10000: train loss 0.383477, train error 0.14
	step 11000: train loss 0.46602, train error 0.175
	step 12000: train loss 0.226437, train error 0.1
	step 13000: train loss 0.218105, train error 0.075
	step 14000: train loss 0.23236, train error 0.1
	step 15000: train loss 0.270004, train error 0.115
	step 16000: train loss 0.275902, train error 0.105
	step 17000: train loss 0.266374, train error 0.12
	step 18000: train loss 0.230

In [4]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=10, num_steps=20000, lr=0.08, keep_prob=0.8, reg=0)

begin training features // num_features: 10, lr: 0.08, reg: 0
	step 0: train loss 32.3909, train error 0.53
	step 1000: train loss 0.741538, train error 0.36
	step 2000: train loss 0.523078, train error 0.24
	step 3000: train loss 0.435838, train error 0.2
	step 4000: train loss 0.397323, train error 0.17
	step 5000: train loss 0.428458, train error 0.185
	step 6000: train loss 0.364371, train error 0.175
	step 7000: train loss 0.385899, train error 0.18
	step 8000: train loss 0.30985, train error 0.105
	step 9000: train loss 0.363459, train error 0.18
	step 10000: train loss 0.300627, train error 0.1
	step 11000: train loss 0.363923, train error 0.15
	step 12000: train loss 0.274563, train error 0.115
	step 13000: train loss 0.275265, train error 0.11
	step 14000: train loss 0.282712, train error 0.125
	step 15000: train loss 0.261695, train error 0.1
	step 16000: train loss 0.258976, train error 0.115
	step 17000: train loss 0.194663, train error 0.065
	step 18000: train loss 0.27735

In [5]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=40, num_steps=20000, lr=0.06, keep_prob=0.8, reg=0)

begin training features // num_features: 40, lr: 0.06, reg: 0
	step 0: train loss 0.973856, train error 0.445
	step 1000: train loss 0.752369, train error 0.325
	step 2000: train loss 0.640358, train error 0.29
	step 3000: train loss 0.490557, train error 0.21
	step 4000: train loss 0.524109, train error 0.195
	step 5000: train loss 0.403861, train error 0.165
	step 6000: train loss 0.390046, train error 0.145
	step 7000: train loss 0.469466, train error 0.2
	step 8000: train loss 0.331546, train error 0.155
	step 9000: train loss 0.340558, train error 0.145
	step 10000: train loss 0.363098, train error 0.145
	step 11000: train loss 0.3749, train error 0.14
	step 12000: train loss 0.243341, train error 0.1
	step 13000: train loss 0.24419, train error 0.1
	step 14000: train loss 0.316556, train error 0.125
end training features // time elapsed: 573.8584 s
test set error: 0.1306 // time elapsed: 0.7965 s
classification accuracy: 0.6765



In [4]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=40, num_steps=20000, lr=0.05, keep_prob=0.8, reg=0)

begin training features // num_features: 40, lr: 0.05, reg: 0
	step 0: train loss 4.37407, train error 0.525
	step 1000: train loss 0.593437, train error 0.25
	step 2000: train loss 0.537254, train error 0.225
	step 3000: train loss 0.692266, train error 0.27
	step 4000: train loss 0.444745, train error 0.175
	step 5000: train loss 0.480615, train error 0.2
	step 6000: train loss 0.546736, train error 0.23
	step 7000: train loss 0.413727, train error 0.17
	step 8000: train loss 0.433921, train error 0.2
	step 9000: train loss 0.348098, train error 0.155
	step 10000: train loss 0.363398, train error 0.135
	step 11000: train loss 0.309652, train error 0.145
	step 12000: train loss 0.303834, train error 0.13
	step 13000: train loss 0.304189, train error 0.125
	step 14000: train loss 0.337917, train error 0.14
	step 15000: train loss 0.321549, train error 0.14
	step 16000: train loss 0.316696, train error 0.13
	step 17000: train loss 0.258314, train error 0.085
	step 18000: train loss 0.29

In [5]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=40, num_steps=20000, lr=0.07, keep_prob=0.8, reg=0)

begin training features // num_features: 40, lr: 0.07, reg: 0
	step 0: train loss 20.7745, train error 0.495
	step 1000: train loss 0.617854, train error 0.245
	step 2000: train loss 0.486558, train error 0.2
	step 3000: train loss 0.540014, train error 0.205
	step 4000: train loss 0.399299, train error 0.16
	step 5000: train loss 0.454911, train error 0.195
	step 6000: train loss 0.524553, train error 0.23
	step 7000: train loss 0.348629, train error 0.135
	step 8000: train loss 0.312973, train error 0.115
	step 9000: train loss 0.294735, train error 0.14
	step 10000: train loss 0.268463, train error 0.115
	step 11000: train loss 0.36601, train error 0.14
	step 12000: train loss 0.329385, train error 0.12
	step 13000: train loss 0.292969, train error 0.12
	step 14000: train loss 0.274964, train error 0.105
	step 15000: train loss 0.215314, train error 0.09
	step 16000: train loss 0.296537, train error 0.12
	step 17000: train loss 0.170805, train error 0.035
	step 18000: train loss 0.1

In [4]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=40, num_steps=20000, lr=0.09, keep_prob=0.8, reg=0)

begin training features // num_features: 40, lr: 0.09, reg: 0
	step 0: train loss 286.387, train error 0.515
	step 1000: train loss 0.635977, train error 0.28
	step 2000: train loss 0.533994, train error 0.255
	step 3000: train loss 0.563628, train error 0.26
	step 4000: train loss 0.445742, train error 0.2
	step 5000: train loss 0.409173, train error 0.175
	step 6000: train loss 0.506018, train error 0.205
	step 7000: train loss 0.480688, train error 0.21
	step 8000: train loss 0.373947, train error 0.175
	step 9000: train loss 0.335161, train error 0.15
	step 10000: train loss 0.31254, train error 0.13
	step 11000: train loss 0.393818, train error 0.155
	step 12000: train loss 0.305246, train error 0.135
	step 13000: train loss 0.336285, train error 0.11
	step 14000: train loss 0.246613, train error 0.08
	step 15000: train loss 0.270262, train error 0.125
	step 16000: train loss 0.314392, train error 0.14
	step 17000: train loss 0.334245, train error 0.145
	step 18000: train loss 0.2

In [5]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=40, num_steps=20000, lr=0.06, keep_prob=0.8, reg=0)

begin training features // num_features: 40, lr: 0.06, reg: 0
	step 0: train loss 15.5351, train error 0.455
	step 1000: train loss 0.719534, train error 0.265
	step 2000: train loss 0.521983, train error 0.21
	step 3000: train loss 0.496413, train error 0.205
	step 4000: train loss 0.388437, train error 0.14
	step 5000: train loss 0.40976, train error 0.195
	step 6000: train loss 0.436177, train error 0.185
	step 7000: train loss 0.473556, train error 0.195
	step 8000: train loss 0.335537, train error 0.165
	step 9000: train loss 0.301007, train error 0.125
	step 10000: train loss 0.407019, train error 0.175
	step 11000: train loss 0.257525, train error 0.11
	step 12000: train loss 0.346975, train error 0.135
	step 13000: train loss 0.298068, train error 0.115
	step 14000: train loss 0.292826, train error 0.12
	step 15000: train loss 0.324778, train error 0.14
	step 16000: train loss 0.192168, train error 0.075
	step 17000: train loss 0.300938, train error 0.11
	step 18000: train loss

In [6]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=40, num_steps=20000, lr=0.06, keep_prob=0.75, reg=0)

begin training features // num_features: 40, lr: 0.06, reg: 0
	step 0: train loss 30.0057, train error 0.505
	step 1000: train loss 0.633051, train error 0.28
	step 2000: train loss 0.567546, train error 0.23
	step 3000: train loss 0.473136, train error 0.2
	step 4000: train loss 0.473553, train error 0.225
	step 5000: train loss 0.469023, train error 0.155
	step 6000: train loss 0.394022, train error 0.19
	step 7000: train loss 0.436986, train error 0.165
	step 8000: train loss 0.513546, train error 0.21
	step 9000: train loss 0.371871, train error 0.16
	step 10000: train loss 0.366107, train error 0.15
	step 11000: train loss 0.393364, train error 0.18
	step 12000: train loss 0.364603, train error 0.15
	step 13000: train loss 0.363729, train error 0.16
	step 14000: train loss 0.292048, train error 0.115
	step 15000: train loss 0.327261, train error 0.145
	step 16000: train loss 0.347884, train error 0.125
	step 17000: train loss 0.218298, train error 0.1
	step 18000: train loss 0.290

In [ ]:
test_model(model=ConvModelMedium, settings=Triplet_Hinge_Settings, 
           data=data['cifar10'], feature_samplers=triplet_samplers['cifar10'],
           classification_samplers=samplers['cifar10'],
           num_features=20, num_steps=20000, lr=0.06, keep_prob=0.75, reg=0)

begin training features // num_features: 20, lr: 0.06, reg: 0
	step 0: train loss 3.57412, train error 0.47
	step 1000: train loss 0.738974, train error 0.335
	step 2000: train loss 0.55301, train error 0.235
	step 3000: train loss 0.502972, train error 0.21
	step 4000: train loss 0.398825, train error 0.185
	step 5000: train loss 0.511164, train error 0.2
	step 6000: train loss 0.413415, train error 0.16
	step 7000: train loss 0.455252, train error 0.2
	step 8000: train loss 0.503895, train error 0.22
	step 9000: train loss 0.401057, train error 0.16
